<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [36]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [ ]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
# with open('../../data/hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1161369
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "usa", "city": "bondville", "street": "bondville-illinois-usa", "aqicn_url": "https://api.waqi.info/feed/@7638", "latitude": 40.11, "longitude": -88.37}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [42]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_new',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
# aq_today_df

df_aq = air_quality_fg.read()

# Check if 'date' column in df_aq is tz-aware or tz-naive
if df_aq['date'].dt.tz is not None:
    # Convert 'date' column in aq_today_df to tz-aware
    aq_today_df['date'] = pd.to_datetime(aq_today_df['date']).dt.tz_localize(df_aq['date'].dt.tz)
else:
    # Convert 'date' column in aq_today_df to tz-naive
    aq_today_df['date'] = pd.to_datetime(aq_today_df['date']).dt.tz_localize(None)

# Ensure the 'date' column is in datetime format
df_aq['date'] = pd.to_datetime(df_aq['date'])

# Set 'date' as the index for both DataFrames
df_aq.set_index('date', inplace=True)
aq_today_df.set_index('date', inplace=True)

# Get today's date from the aq_today_df DataFrame
today_date = aq_today_df.index[0]

# Try to access today's date row directly; if it doesn't exist, append
try:
    # If today_date exists, overwrite the 'pm25' value
    df_aq.loc[today_date, 'pm25'] = aq_today_df.loc[today_date, 'pm25']
    print('Date already exists in the DataFrame, value updated.')
except KeyError:
    # If today_date does not exist, concatenate
    df_aq = pd.concat([df_aq, aq_today_df])
    print('Date did not exist in the DataFrame, row appended.')

# Sort the DataFrame by the 'date' column
df_aq.sort_index(inplace=True)

# Reindex the DataFrame to include all dates in the range, filling missing dates with NaN values
date_range = pd.date_range(start=df_aq.index.min(), end=df_aq.index.max())
df_aq = df_aq.reindex(date_range).rename_axis('date')

# Compute the rolling mean of the last 3 days, allowing for NaN values
df_aq['rolling_mean_pm25'] = df_aq['pm25'].rolling(window=3, min_periods=1).mean()

# Reset the index if needed
df_aq.reset_index(inplace=True)

# Drop rows with NaN values
df_aq.dropna(inplace=True)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.21s) 
Date already exists in the DataFrame, value updated.


In [57]:
df_aq

,date,pm25,rolling_mean_pm25,country,city,street,url
0,2015-01-29 00:00:00+00:00,66.0,66.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
1,2015-01-30 00:00:00+00:00,16.0,41.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
2,2015-01-31 00:00:00+00:00,46.0,42.666667,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3,2015-02-01 00:00:00+00:00,53.0,38.333333,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
4,2015-02-02 00:00:00+00:00,22.0,40.333333,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
...,...,...,...,...,...,...,...
3568,2024-11-05 00:00:00+00:00,17.0,19.333333,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3569,2024-11-06 00:00:00+00:00,13.0,16.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3570,2024-11-07 00:00:00+00:00,18.0,16.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3571,2024-11-08 00:00:00+00:00,31.0,20.666667,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638


In [28]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   pm25     1 non-null      float32            
 1   country  1 non-null      object             
 2   city     1 non-null      object             
 3   street   1 non-null      object             
 4   date     1 non-null      datetime64[ns, UTC]
 5   url      1 non-null      object             
dtypes: datetime64[ns, UTC](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [6]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 40.0°N -88.25°E
Elevation 221.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-14,11.10,0.0,15.188415,264.559753,bondville
1,2024-11-15,8.80,0.0,8.217153,298.810699,bondville
2,2024-11-16,4.40,0.0,7.653705,131.185822,bondville
3,2024-11-17,9.00,0.0,14.182355,203.962494,bondville
4,2024-11-18,9.00,0.0,9.957108,49.398785,bondville
5,2024-11-19,14.10,0.3,26.208397,142.815216,bondville
6,2024-11-20,7.00,0.0,6.369050,317.290619,bondville
7,2024-11-21,10.05,0.2,22.545315,70.407631,bondville
8,2024-11-22,6.40,0.0,22.565422,213.943634,bondville
9,2024-11-23,4.15,0.0,10.661107,258.310669,bondville


In [7]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [30]:
# Insert new data row, containing todays reading and also last 3 day rolling mean
air_quality_fg.insert(df_aq.tail(1))

2024-11-14 19:22:00,980 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161369/fs/1152072/fg/1347992


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_new_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161369/jobs/named/air_quality_new_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17430ff40>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 671795
         }
       },
       "result": {
         "observed_value": 36.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-14T06:22:00.000980Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [31]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-14 19:22:12,439 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161369/fs/1152072/fg/1342814


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161369/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17515ebc0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 666639
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-14T06:22:12.000439Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
